In [1]:
import tensorflow as tf
from collections import defaultdict
import gzip
import cv2 as cv
import os
import numpy as np
import random

In [8]:
def extract_data(filepath):
    tr_data1 = []
    tr_data2 = []
    tr_data3 = []
    te_data1 = []
    te_data2 = []
    te_data3 = []
    tr_data_labels = []
    te_data_labels = []
    
    IMG_SIZE = 100
    IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
    vgg = tf.keras.applications.VGG16(input_shape=IMG_SHAPE, weights="imagenet", include_top=False)
            
    for label in os.listdir(filepath):
        label_path = os.path.join(filepath, label)

        for files in os.listdir(label_path):
            instances_path = os.path.join(label_path, files)

            if str(files) == 'test':
                for instance in os.listdir(instances_path):
                    images_path = os.path.join(instances_path, instance)
                    te_data_labels.append(label)
                    for img in os.listdir(images_path):
                        image_path = os.path.join(images_path, img)
                        if int(img[0:-4]) == 1:
                            image = cv.imread(image_path)
                            re_image = cv.resize(image, (IMG_SIZE, IMG_SIZE))
                            te_data1.append(re_image)
                        elif int(img[0:-4]) == 6:
                            image = cv.imread(image_path)
                            re_image = cv.resize(image, (IMG_SIZE, IMG_SIZE))
                            te_data2.append(re_image)
                        elif int(img[0:-4]) == 10:
                            image = cv.imread(image_path)
                            re_image = cv.resize(image, (IMG_SIZE, IMG_SIZE))
                            te_data3.append(re_image)
                    
            elif str(files) == 'train':
                for instance in os.listdir(instances_path):
                    images_path = os.path.join(instances_path, instance)
                    tr_data_labels.append(label)
                    for img in os.listdir(images_path):
                        image_path = os.path.join(images_path, img)
                        if int(img[0:-4]) == 1:
                            image = cv.imread(image_path)
                            re_image = cv.resize(image, (IMG_SIZE, IMG_SIZE))
                            tr_data1.append(re_image)              
                        elif int(img[0:-4]) == 6:
                            image = cv.imread(image_path)
                            re_image = cv.resize(image, (IMG_SIZE, IMG_SIZE))
                            tr_data2.append(re_image)
                        elif int(img[0:-4]) == 10:
                            image = cv.imread(image_path)
                            re_image = cv.resize(image, (IMG_SIZE, IMG_SIZE))
                            tr_data3.append(re_image)          
                
    tr_data1 = np.array(tr_data1, dtype="float") / 255.0
    tr_data2 = np.array(tr_data2, dtype="float") / 255.0
    tr_data3 = np.array(tr_data3, dtype="float") / 255.0
    te_data1 = np.array(te_data1, dtype="float") / 255.0
    te_data2 = np.array(te_data2, dtype="float") / 255.0
    te_data3 = np.array(te_data3, dtype="float") / 255.0
    
    tr_features = vgg.predict(tr_data1, batch_size=32)
    tr_features_flatten1 = tr_features.reshape((tr_features.shape[0], 3 * 3 * 512))
    
    tr_features = vgg.predict(tr_data2, batch_size=32)
    tr_features_flatten2 = tr_features.reshape((tr_features.shape[0], 3 * 3 * 512))
    
    tr_features = vgg.predict(tr_data3, batch_size=32)
    tr_features_flatten3 = tr_features.reshape((tr_features.shape[0], 3 * 3 * 512))
    
    te_features = vgg.predict(te_data1, batch_size=32)
    te_features_flatten1 = te_features.reshape((te_features.shape[0], 3 * 3 * 512))

    te_features = vgg.predict(te_data2, batch_size=32)
    te_features_flatten2 = te_features.reshape((te_features.shape[0], 3 * 3 * 512))
    
    te_features = vgg.predict(te_data3, batch_size=32)
    te_features_flatten3 = te_features.reshape((te_features.shape[0], 3 * 3 * 512))
    
    # data = data.reshape(data.shape[0], 50, 50, 3)
    tr_data_labels = np.array(tr_data_labels)
    te_data_labels = np.array(te_data_labels)
    label_names = np.unique(tr_data_labels)
    # data_labels = tf.one_hot(indices=data_labels, depth=10)
    print(features_flatten.shape)
    return tr_features_flatten1, tr_features_flatten2, tr_features_flatten3, tr_data_labels, \
            te_features_flatten1, te_features_flatten2, te_features_flatten3, te_data_labels, label_names

In [9]:
if __name__ == "__main__":
    
    # lesser categories for faster loading during testing the code    
    data_path = '../ModelNet40/image_dataset'

    # For easier disk read operation
    np_train_path1 = '../ModelNet40/numpy_dataset/train1.npy.gz'
    np_train_path2 = '../ModelNet40/numpy_dataset/train2.npy.gz'
    np_train_path3 = '../ModelNet40/numpy_dataset/train3.npy.gz'
    np_test_path1 = '../ModelNet40/numpy_dataset/test1.npy.gz'
    np_test_path2 = '../ModelNet40/numpy_dataset/test2.npy.gz'
    np_test_path3 = '../ModelNet40/numpy_dataset/test3.npy.gz'
    np_train_label_path = '../ModelNet40/numpy_dataset/train_labels.npy'
    np_test_label_path = '../ModelNet40/numpy_dataset/test_labels.npy'
    np_label_name_path = '../ModelNet40/numpy_dataset/label_names.npy'

    print("\n..........loading dataset from disk..........\n")
    train_data1, train_data2, train_data3, train_labels, \
    test_data1, test_data2, test_data3, test_labels, label_names = extract_data(data_path)

    os.makedirs(os.path.dirname(np_train_path), exist_ok=True)

    with gzip.GzipFile(np_train_path1, "w") as f:
        np.save(f, train_data1)
    with gzip.GzipFile(np_train_path2, "w") as f:
        np.save(f, train_data2)
    with gzip.GzipFile(np_train_path3, "w") as f:
        np.save(f, train_data3)
    with gzip.GzipFile(np_test_path1, "w") as f:
        np.save(f, test_data1)
    with gzip.GzipFile(np_test_path2, "w") as f:
        np.save(f, test_data2)
    with gzip.GzipFile(np_test_path3, "w") as f:
        np.save(f, test_data3)

    np.save(np_train_label_path, train_labels)
    np.save(np_test_label_path, test_labels)
    np.save(np_label_name_path, label_names)


..........loading dataset from disk..........



ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/sudhakaran/Desktop/3d_recognition_thesis/2d_recognition/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-f34d38e723f5>", line 19, in <module>
    test_data1, test_data2, test_data3, test_labels, label_names = extract_data(data_path)
  File "<ipython-input-8-250346c46f4c>", line 69, in extract_data
    tr_features = vgg.predict(tr_data2, batch_size=32)
  File "/home/sudhakaran/Desktop/3d_recognition_thesis/2d_recognition/venv/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py", line 1113, in predict
    self, x, batch_size=batch_size, verbose=verbose, steps=steps)
  File "/home/sudhakaran/Desktop/3d_recognition_thesis/2d_recognition/venv/lib/python3.6/site-packages/tensorflow/python/keras/engine/training_arrays.py", line 312, in model_iteration
    ins_batch = slice_arrays(ins, batch_ids)
  File "/ho

KeyboardInterrupt: 